In [1]:
import pandas as pd
import numpy as np
import json
import os
import requests
from config import api_key, postgres_pass#, heroku_pass, heroku_URI
import yfinance as yf
from sklearn.preprocessing import StandardScaler
import joblib
import tensorflow as tf
import keras

In [3]:
#Select Ticker
ticker = 'msft'
tick = yf.Ticker(ticker)
# pull Ticker details from Yahoo Finance
history = tick.history(period="max")

In [ ]:
# Yahoo API

#ticvker = 'kim'
#endpoint = "/v11/finance/quoteSummary/"
#url = "https://yfapi.net" + endpoint + ticker
#querystring = {"modules":"defaultKeyStatistics"}

#headers = {
#    'x-api-key': api_key
#    }

#response = requests.request("GET", url, headers=headers, params=querystring)

#tick_resp = response
#tick_resp.json()['quoteSummary']['result'][0]['defaultKeyStatistics']

In [6]:
history.isnull().sum()

Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

In [7]:
len(history)

9084

In [8]:
history.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')

In [9]:
history.dtypes

Open            float64
High            float64
Low             float64
Close           float64
Volume            int64
Dividends       float64
Stock Splits    float64
dtype: object

In [11]:
history.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,9084.000000,9084.000000,9084.000000,9084.000000,9.084000e+03,9084.000000,9084.000000
mean,36.275610,36.648222,35.891539,36.285559,5.874657e+07,0.002482,0.001871
std,60.026531,60.604230,59.406873,60.043727,3.845175e+07,0.042193,0.059788
min,0.055949,0.058143,0.055949,0.057046,2.304000e+06,0.000000,0.000000
25%,2.560896,2.599618,2.546086,2.577866,3.460562e+07,0.000000,0.000000
50%,18.941180,19.143998,18.735768,18.948545,5.203020e+07,0.000000,0.000000
75%,29.290971,29.603001,28.892224,29.244802,7.265020e+07,0.000000,0.000000
max,343.908878,348.948476,341.493913,342.402008,1.031789e+09,3.000000,2.000000


In [12]:
history.to_csv('history_data.csv', index=False)

In [14]:
df_hist = pd.read_csv('history_data.csv')
df_hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,0.055949,0.064177,0.055949,0.061434,1031788800,0.0,0.0
1,0.061434,0.064725,0.061434,0.063628,308160000,0.0,0.0
2,0.063628,0.065274,0.063628,0.064725,133171200,0.0,0.0
3,0.064725,0.065274,0.062531,0.063079,67766400,0.0,0.0
4,0.063079,0.063628,0.061434,0.061982,47894400,0.0,0.0
...,...,...,...,...,...,...,...
9079,298.890015,300.140015,294.899994,299.160004,28351200,0.0,0.0
9080,299.799988,305.000000,298.769989,304.059998,27599700,0.0,0.0
9081,300.510010,303.230011,297.720001,299.489990,25715400,0.0,0.0
9082,299.140015,304.200012,298.320007,304.100006,24484500,0.0,0.0
